# 1. Prepare NLPText

In [1]:
from pprint import pprint
from nlptext.base import BasicObject

########### ResumeNER ###########
CORPUSPath = 'corpus/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW)

from nlptext.corpus import Corpus
corpus = Corpus()
# corpus.IdxFolderStartEnd

print([i for i in corpus.TOKEN])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


'File'
corpus/ResumeNER/test.char.bmes


Loading model cost 0.591 seconds.
Prefix dict has been built succesfully.


corpus/ResumeNER/train.char.bmes
corpus/ResumeNER/dev.char.bmes
Total Num of All    Tokens 149812
The Total Number of Tokens: 149812
Counting the number unique Tokens...          	 2019-06-20 18:59:35.541679
		Done!
Generating Dictionary of Token Unique...	 2019-06-20 18:59:35.603179
		The length of DTU is: 1891 	 2019-06-20 18:59:35.603574
Generating the ORIGTokenIndex...       	 2019-06-20 18:59:35.603643
		The idx of token is: 0 	 2019-06-20 18:59:35.603793
		Done!
Only Keep First 3500000 Tokens.
The coverage rate is: 0.0
Total Num of Unique Tokens 1891
['</pad>', '</start>', '</end>', 'O', 'CONT-B', 'CONT-E', 'CONT-I', 'CONT-S', 'EDU-B', 'EDU-E', 'EDU-I', 'EDU-S', 'LOC-B', 'LOC-E', 'LOC-I', 'LOC-S', 'NAME-B', 'NAME-E', 'NAME-I', 'NAME-S', 'ORG-B', 'ORG-E', 'ORG-I', 'ORG-S', 'PRO-B', 'PRO-E', 'PRO-I', 'PRO-S', 'RACE-B', 'RACE-E', 'RACE-I', 'RACE-S', 'TITLE-B', 'TITLE-E', 'TITLE-I', 'TITLE-S']
CORPUS	it is Dumped into file: data/ResumeNER/char/Token1891/Pyramid/CORPUS.p
CORPUS	the le

In [2]:
CHANNEL_SETTINGS_TEMPLATE = {
    # CTX_IND
    'token':   {'use': True,'Max_Ngram': 1,}, # always the char-level token
    'basic':   {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'medical': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'radical': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'subcomp': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'stroke':  {'use': False,'Max_Ngram': 1, 'end_grain': False},
    # CTX_DEP
    'pos':     {'use': False, 'tagScheme': 'BIOE',},
    # ANNO
    'annoE':   {'use': True, 'tagScheme': 'BIOE',},
}

BasicObject.BUILD_GRAIN_UNI_AND_LOOKUP(CHANNEL_SETTINGS_TEMPLATE=CHANNEL_SETTINGS_TEMPLATE)


Deal with the Channel: token
Current Channel is        	 token
Current Channel Max_Ngram 	 1
Deal with the Channel: basic
Current Channel is        	 basic
Current Channel Max_Ngram 	 1
		Build Grain Uniqe and LookUp Table for channel: basic
For channel: | basic | build GrainUnique and LookUp
		Write to: data/ResumeNER/char/Token1891/GrainUnique/basic.voc
		Write to: data/ResumeNER/char/Token1891/GrainUnique/basic.tsv
		Write to: data/ResumeNER/char/Token1891/GrainUnique/basic.lkp
Deal with the Channel: annoE
Current Channel is        	 annoE
Current Channel Max_Ngram 	 1
		Build GrainUnique for channel: annoE-e
annoE 1 False BIOE
		Write to: data/ResumeNER/char/Token1891/GrainUnique/annoE-e.voc
		Write to: data/ResumeNER/char/Token1891/GrainUnique/annoE-e.tsv


In [3]:
def load_anno(BasicObject, channel, tagScheme):
    # channel_anno = 'annoE'
    GU = BasicObject.getGrainUnique(anno_field, tagScheme = tagScheme)
    tags = GU[0]
    tag_size = len(tags)
    labels = list(set([i.split('-')[0] for i in tags if '-' in i]))
    labels.sort()
    return labels, tags, tag_size

anno_field = 'annoE'
tagScheme  = 'BIO' # currently, only BIO is available, this could be updated
labels, tags, tag_size = load_anno(BasicObject, anno_field, tagScheme)
print(labels)
print(tags)
print(tag_size)

		Build GrainUnique for channel: annoE
annoE 1 False BIO
		Write to: data/ResumeNER/char/Token1891/GrainUnique/annoE.voc
		Write to: data/ResumeNER/char/Token1891/GrainUnique/annoE.tsv
['CONT', 'EDU', 'LOC', 'NAME', 'ORG', 'PRO', 'RACE', 'TITLE']
['</pad>', '</start>', '</end>', 'CONT-B', 'CONT-I', 'EDU-B', 'EDU-I', 'LOC-B', 'LOC-I', 'NAME-B', 'NAME-I', 'O', 'ORG-B', 'ORG-I', 'PRO-B', 'PRO-I', 'RACE-B', 'RACE-I', 'TITLE-B', 'TITLE-I']
20


# 2. Featurize a Sentence

In [4]:
from nlptext.sentence import Sentence
sent = Sentence(12)
print(sent.Idx)
print(sent)
print(sent.sentence)
Channel_Settings = BasicObject.CHANNEL_SETTINGS
print(Channel_Settings)

12
<st 12 (tokenNum: 22) >
1965年10月出生,本科学历,高级会计师。
{'token': {'Max_Ngram': 1}, 'basic': {'Max_Ngram': 1, 'end_grain': False}, 'annoE': {'tagScheme': 'BIOE'}}


## 2.1 String Features

In [5]:
from crfpp.crftools import get_sent_strfeats

get_sent_strfeats(sent, Channel_Settings) # this need some attention

,token_0,basic_0,annoE_0
0,1,NUM,O
1,9,NUM,O
2,6,NUM,O
3,5,NUM,O
4,年,CHN,O
5,1,NUM,O
6,0,NUM,O
7,月,CHN,O
8,出,CHN,O
9,生,CHN,O


## 2.2 Vector Features (TODO)

In [6]:
from crfpp.crftools import get_sent_vecfeats

get_sent_vecfeats(sent, Channel_Settings, fieldembed = None)

# 3. Featurize all Sentences

In [7]:
BasicObject

nlptext.base.BasicObject

## 3.1 Featurize all Sentences with String

In [8]:
from crfpp.crftools import featurize_nlptext_sentences
sents = featurize_nlptext_sentences(BasicObject, feat_type = 'str')

		Build GrainUnique for channel: pos
pos 1 False BIO
		Write to: data/ResumeNER/char/Token1891/GrainUnique/pos.voc
		Write to: data/ResumeNER/char/Token1891/GrainUnique/pos.tsv
Save Featurized Sentences to:
	 data/ResumeNER/char/Token1891/Pyramid/_Feat_SENT_Str.crfpp


## 3.2 Featurize all Sentences with Vector (TODO)

In [9]:
# from crfpp.crftools import featurize_nlptext_sentences
# sents = featurize_nlptext_sentences(BasicObject, feat_type = 'vec')

# 4. Generate Template

The `template` is used to convert `input feats` to `derivative feats`.

In [10]:
import pandas as pd
import numpy as np

# individaul

individual_para = {
    1: 3,
    2: 2,
    3: 1,
}

def generate_template(input_feats_num = 5, individual_para = individual_para, path = None):
    '''
     this still needs more consideration
    '''
    if not path:
        path = '_template'
        
    L = ['# Unigram\n\n']
    fld_idx = 0
    for feat_idx in range(input_feats_num):
        for gram_num, window_size in individual_para.items():
            if gram_num == 1:
                for token_i in range(-window_size, window_size + 1):
                    L.append('U{}:%x[{},{}]\n'.format(str(fld_idx), str(token_i), str(feat_idx)))
                    fld_idx = fld_idx + 1
            if gram_num == 2 and feat_idx <= 5:
                for token_i in range(-window_size, window_size + 1):
                    L.append('U{}:%x[{},{}]/%x[{},{}]\n'.format(str(fld_idx), 
                                                                str(token_i), str(feat_idx), 
                                                                str(token_i + 1), str(feat_idx)))
                    fld_idx = fld_idx + 1
                    
            if gram_num == 3 and feat_idx <= 5:
                for token_i in range(-window_size, window_size + 1):
                    L.append('U{}:%x[{},{}]/%x[{},{}]/%x[{},{}]\n'.format(str(fld_idx), 
                                                                          str(token_i - 1), str(feat_idx), 
                                                                          str(token_i), str(feat_idx), 
                                                                          str(token_i + 1), str(feat_idx)))
                    fld_idx = fld_idx + 1
    
    
    L.append('\n\n# Bigram\nB')
    
    with open(path, 'w') as f:
        f.write(''.join(L))
        
    return ''.join(L)

print(generate_template(input_feats_num = 20))

# Unigram

U0:%x[-3,0]
U1:%x[-2,0]
U2:%x[-1,0]
U3:%x[0,0]
U4:%x[1,0]
U5:%x[2,0]
U6:%x[3,0]
U7:%x[-2,0]/%x[-1,0]
U8:%x[-1,0]/%x[0,0]
U9:%x[0,0]/%x[1,0]
U10:%x[1,0]/%x[2,0]
U11:%x[2,0]/%x[3,0]
U12:%x[-2,0]/%x[-1,0]/%x[0,0]
U13:%x[-1,0]/%x[0,0]/%x[1,0]
U14:%x[0,0]/%x[1,0]/%x[2,0]
U15:%x[-3,1]
U16:%x[-2,1]
U17:%x[-1,1]
U18:%x[0,1]
U19:%x[1,1]
U20:%x[2,1]
U21:%x[3,1]
U22:%x[-2,1]/%x[-1,1]
U23:%x[-1,1]/%x[0,1]
U24:%x[0,1]/%x[1,1]
U25:%x[1,1]/%x[2,1]
U26:%x[2,1]/%x[3,1]
U27:%x[-2,1]/%x[-1,1]/%x[0,1]
U28:%x[-1,1]/%x[0,1]/%x[1,1]
U29:%x[0,1]/%x[1,1]/%x[2,1]
U30:%x[-3,2]
U31:%x[-2,2]
U32:%x[-1,2]
U33:%x[0,2]
U34:%x[1,2]
U35:%x[2,2]
U36:%x[3,2]
U37:%x[-2,2]/%x[-1,2]
U38:%x[-1,2]/%x[0,2]
U39:%x[0,2]/%x[1,2]
U40:%x[1,2]/%x[2,2]
U41:%x[2,2]/%x[3,2]
U42:%x[-2,2]/%x[-1,2]/%x[0,2]
U43:%x[-1,2]/%x[0,2]/%x[1,2]
U44:%x[0,2]/%x[1,2]/%x[2,2]
U45:%x[-3,3]
U46:%x[-2,3]
U47:%x[-1,3]
U48:%x[0,3]
U49:%x[1,3]
U50:%x[2,3]
U51:%x[3,3]
U52:%x[-2,3]/%x[-1,3]
U53:%x[-1,3]/%x[0,3]
U54:%x[0,3]/%x[1,3]
U55:%x[1,3]/%x[2,3

# 5. Load Data

In [11]:
print(len(sents))

4617


In [12]:
from crfpp.loaddata import loadData

seed = 10
cross_validation = False

cross_idx = 0
cross_num = 5

trainSents, testSents = loadData(sents, cross_num, seed = seed, cross_validation = cross_validation, cross_idx = cross_idx)

print(len(trainSents), len(testSents))

3694 923


# 6. Train Model

In [13]:
from nlptext.utils.channel import getChannelName

def get_model_name(BasicObject, Channel_Settings):
    return BasicObject.TokenNum_Dir.replace('data/', 'model/') + '/' + "_".join([getChannelName(ch, style = 'abbr') for ch in Channel_Settings])
# Channel_Settings
model = BasicObject.TokenNum_Dir.replace('data/', 'model/') + '/' + "_".join([getChannelName(ch, style = 'abbr') for ch in Channel_Settings])
model

'model/ResumeNER/char/Token1891/T_b_E'

In [15]:
import os
from datetime import datetime
from crfpp.crftools import crf_learn
import pickle

def crfpp_train(model, trainSents, Channel_Settings, feat_type = 'str'):
    
    if 'str' in feat_type.lower():
        get_sent_feats = get_sent_strfeats
    elif 'vec' in feat_type.lower():
        get_sent_feats = get_sent_vecfeats
        
    tmp_dir = model.replace('model', '_tmp')
    
    if not os.path.exists(tmp_dir):
        os.makedirs(tmp_dir)
        
    if not os.path.exists(model):
        os.makedirs(model)
        
    model_path = model + '/model'
    para_path  = model + '/para.p'
    template_path = model + '/template'
    feats_data_path = tmp_dir + '/feats.txt'
    with open(para_path, 'wb') as f:
        pickle.dump(Channel_Settings, f)
    
    DFtrain = []
    for idx, sent in enumerate(trainSents):
        if idx % 500 == 0:
            print(datetime.now(), idx)
        df = get_sent_feats(sent, Channel_Settings)
        df.loc[len(df)] = np.NaN     ## Trick Here
        DFtrain.append(df)           ## Trick Here
    DFtrain = pd.concat(DFtrain).reset_index(drop=True)
    DFtrain.to_csv(feats_data_path, sep = '\t', encoding = 'utf=8', header = False, index = False )
    
    generate_template(input_feats_num = DFtrain.shape[1] - 1, path = template_path) 
    crf_learn(feats_data_path, model_path, template_path  = template_path)
    
    return None

In [16]:
crfpp_train(model, trainSents, Channel_Settings)

2019-06-20 19:00:10.736289 0
2019-06-20 19:00:12.027778 500
2019-06-20 19:00:13.252859 1000
2019-06-20 19:00:14.541602 1500
2019-06-20 19:00:15.787799 2000
2019-06-20 19:00:17.006271 2500
2019-06-20 19:00:18.229760 3000
2019-06-20 19:00:19.444364 3500
CRF++: Yet Another CRF Tool Kit
Copyright (C) 2005-2013 Taku Kudo, All rights reserved.

reading training data: 100.. 200.. 300.. 400.. 500.. 600.. 700.. 800.. 900.. 1000.. 1100.. 1200.. 1300.. 1400.. 1500.. 1600.. 1700.. 1800.. 1900.. 2000.. 2100.. 2200.. 2300.. 2400.. 2500.. 2600.. 2700.. 2800.. 2900.. 3000.. 3100.. 3200.. 3300.. 3400.. 3500.. 3600.. 
Done!1.11 s

Number of sentences: 3694
Number of features:  1313165
Number of thread(s): 8
Freq:                2
eta:                 0.00010
C:                   5.00000
shrinking size:      20
iter=0 terr=0.99792 serr=1.00000 act=1313165 obj=342515.66080 diff=1.00000
iter=1 terr=0.54871 serr=1.00000 act=1313165 obj=218904.28771 diff=0.36089
iter=2 terr=0.37783 serr=1.00000 act=1313165 o

# 7. Tagger a Sentence

In [17]:
sent = testSents[13]
print(sent)
print(sent.sentence)
print(model)

<st 63 (tokenNum: 84) >
2000年11月至2003年2月任北京航空航天大学副书记兼副校长,现任北京航空航天大学常务副校长,国家863计划计算机技术主题首席科学家,国家信息化专家咨询委员会成员。
model/ResumeNER/char/Token1891/T_b_E


In [18]:
from crfpp.crftools import get_sent_strfeats, crf_test
from crfpp.evals import read_target_seq, extractSET

def tagger(model, sent, Channel_Settings = None):
    '''
        basically from crf_test
        sent: a sentence, could be without annotation
    '''
    if not Channel_Settings:
        with open(model + '/para.p', 'rb') as f:
            Channel_Settings = pickle.load(f)
    # 1. get sentence feats
    # hopefully, the model_config is included in model_path
    feats_data_path   = '_tmp/_tagger_feats.txt'
    results_data_path = '_tmp/_tagger_results.txt'
    model_path = model + '/model'

    # get Channel_Settings
    # get use sent strfeats or sent vecfeats settings
    df = get_sent_strfeats(sent, Channel_Settings, train = False)
    df.to_csv(feats_data_path, sep = '\t', encoding = 'utf=8', header = False, index = False )
    # 2. save the sentence feats to a file
    
    # 3. tag a sentence
    crf_test(feats_data_path, model_path, results_data_path)

    # 4. read and parse the result to pred_SSET
    # get a tag_seq
    # list of tuple (score, result)
    tag_seq  = read_target_seq(results_data_path)
    pred_SET = extractSET(tag_seq)
    return pred_SET

tagger(model, sent) 

[(17, 25, 'ORG'),
 (25, 28, 'TITLE'),
 (29, 32, 'TITLE'),
 (35, 43, 'ORG'),
 (43, 48, 'TITLE'),
 (49, 59, 'ORG'),
 (59, 68, 'TITLE'),
 (69, 81, 'ORG'),
 (81, 83, 'TITLE')]

# 8. Test Model

In [19]:
print(labels)
print(model)

['CONT', 'EDU', 'LOC', 'NAME', 'ORG', 'PRO', 'RACE', 'TITLE']
model/ResumeNER/char/Token1891/T_b_E


In [20]:
from crfpp.evals import get_sent_annoSET, match_anno_pred_result, calculate_F1_Score, logError


def crfpp_test(model, testSents, Channel_Settings,  labels):
    '''
        sents: a list of sents
        This function could be updated in the future for a better performance.
        But currently, just leave it now.
    '''
    pred_entities = []
    anno_entities = []
    log_list      = []
    # here sents are a batch of sents, not necessary to be all the sents
    # this could be chanage, but it will cause some time, so just ignore it.
    for idx, sent in enumerate(testSents):
        if idx % 200 == 0:
            print(datetime.now(), idx)
        # 200/13s
        pred_SET = tagger(model, sent, Channel_Settings = Channel_Settings)
        pred_entities.append(pred_SET)
        
        anno_SET = get_sent_annoSET(sent)
        anno_entities.append(anno_SET)
        
        error = logError(sent, pred_SET, anno_SET)
        log_list.append(error)
    # return anno_entities, pred_entities
    Result = match_anno_pred_result(anno_entities, pred_entities, labels = labels)
    # return Result
    R = calculate_F1_Score(Result, labels)

    LogError = pd.concat(log_list).reset_index(drop = True)
    return R, LogError
R, LogError = crfpp_test(model, testSents, Channel_Settings,  labels)
R

2019-06-20 19:07:15.959821 0
2019-06-20 19:07:28.811686 200
2019-06-20 19:07:41.666097 400
2019-06-20 19:07:54.516089 600
2019-06-20 19:08:07.411994 800


,Anno,Pred,Match,R,P,F1
CONT,70,68,68,0.971429,1.000000,0.985507
EDU,207,209,199,0.961353,0.952153,0.956731
LOC,8,7,7,0.875000,1.000000,0.933333
NAME,241,235,235,0.975104,1.000000,0.987395
ORG,1115,1107,1021,0.915695,0.922313,0.918992
PRO,64,66,59,0.921875,0.893939,0.907692
RACE,18,17,17,0.944444,1.000000,0.971429
TITLE,1500,1499,1411,0.940667,0.941294,0.940980
E,3223,3208,3017,0.936084,0.940461,0.938268


# 9. Overall Workflow

In [1]:
from pprint import pprint
from nlptext.base import BasicObject

########### ResumeNER ###########
CORPUSPath = 'corpus/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW)

CHANNEL_SETTINGS_TEMPLATE = {
    # CTX_IND
    'token':   {'use': True,'Max_Ngram': 1,}, # always the char-level token
    'basic':   {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'medical': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'radical': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'subcomp': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'stroke':  {'use': False,'Max_Ngram': 1, 'end_grain': False},
    # CTX_DEP
    'pos':     {'use': False, 'tagScheme': 'BIO',},
    # ANNO
    'annoE':   {'use': True, 'tagScheme': 'BIO',},
}

BasicObject.BUILD_GRAIN_UNI_AND_LOOKUP(CHANNEL_SETTINGS_TEMPLATE=CHANNEL_SETTINGS_TEMPLATE)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


'File'
corpus/ResumeNER/test.char.bmes


Loading model cost 0.585 seconds.
Prefix dict has been built succesfully.


corpus/ResumeNER/train.char.bmes
corpus/ResumeNER/dev.char.bmes
Total Num of All    Tokens 149812
The Total Number of Tokens: 149812
Counting the number unique Tokens...          	 2019-06-20 19:37:49.636243
		Done!
Generating Dictionary of Token Unique...	 2019-06-20 19:37:49.685311
		The length of DTU is: 1891 	 2019-06-20 19:37:49.685757
Generating the ORIGTokenIndex...       	 2019-06-20 19:37:49.685829
		The idx of token is: 0 	 2019-06-20 19:37:49.686006
		Done!
Only Keep First 3500000 Tokens.
The coverage rate is: 0.0
Total Num of Unique Tokens 1891
['</pad>', '</start>', '</end>', 'O', 'CONT-B', 'CONT-E', 'CONT-I', 'CONT-S', 'EDU-B', 'EDU-E', 'EDU-I', 'EDU-S', 'LOC-B', 'LOC-E', 'LOC-I', 'LOC-S', 'NAME-B', 'NAME-E', 'NAME-I', 'NAME-S', 'ORG-B', 'ORG-E', 'ORG-I', 'ORG-S', 'PRO-B', 'PRO-E', 'PRO-I', 'PRO-S', 'RACE-B', 'RACE-E', 'RACE-I', 'RACE-S', 'TITLE-B', 'TITLE-E', 'TITLE-I', 'TITLE-S']
CORPUS	it is Dumped into file: data/ResumeNER/char/Token1891/Pyramid/CORPUS.p
CORPUS	the le

In [2]:
from crfpp.crftools import load_anno

anno_field = 'annoE'
Channel_Settings, tagScheme, labels, tags, tag_size = load_anno(BasicObject, anno_field)
print(Channel_Settings)
print(anno_field)
print(tagScheme)
print(labels)
print(tags)
print(tag_size)

{'token': {'Max_Ngram': 1}, 'basic': {'Max_Ngram': 1, 'end_grain': False}, 'annoE': {'tagScheme': 'BIO'}}
annoE
BIO
['CONT', 'EDU', 'LOC', 'NAME', 'ORG', 'PRO', 'RACE', 'TITLE']
['</pad>', '</start>', '</end>', 'CONT-B', 'CONT-I', 'EDU-B', 'EDU-I', 'LOC-B', 'LOC-I', 'NAME-B', 'NAME-I', 'O', 'ORG-B', 'ORG-I', 'PRO-B', 'PRO-I', 'RACE-B', 'RACE-I', 'TITLE-B', 'TITLE-I']
20


In [3]:
from nlptext.utils.channel import getChannelName

def get_model_name(BasicObject, Channel_Settings):
    return BasicObject.TokenNum_Dir.replace('data/', 'model/') + '/' + "_".join([getChannelName(ch, style = 'abbr') for ch in Channel_Settings if 'anno' not in ch])
# Channel_Settings

model = get_model_name(BasicObject, Channel_Settings)
print(model)

model/ResumeNER/char/Token1891/T_b


In [4]:
from nlptext.corpus import Corpus
corpus = Corpus()
sents = corpus.Sentences

print(model)
print(len(sents))
print(Channel_Settings)

cross_num = 4
cross_validation = True

model/ResumeNER/char/Token1891/T_b
4617
{'token': {'Max_Ngram': 1}, 'basic': {'Max_Ngram': 1, 'end_grain': False}, 'annoE': {'tagScheme': 'BIO'}}


In [ ]:
from crfpp.train import train

Performance = train(model, sents, Channel_Settings, labels, cross_num, cross_validation = None, seed = 10)

Performance

2019-06-20 19:37:50.172807 0
2019-06-20 19:37:51.436063 500
2019-06-20 19:37:52.785359 1000
2019-06-20 19:37:54.172588 1500
2019-06-20 19:37:55.395517 2000
2019-06-20 19:37:56.688992 2500
2019-06-20 19:37:57.922568 3000
